Para realizar um **SELECT** em uma tabela **Delta** no Databricks usando **PySpark (Python)**, você pode usar a API SQL ou DataFrame do Spark. Aqui está o processo:

---

### **1. Consulta Usando SQL via PySpark**
Se a tabela foi criada com o comando `%sql CREATE TABLE`, ela está registrada no catálogo do Databricks (Unity Catalog ou Hive Metastore). Você pode consultar diretamente usando SQL dentro de Python.

#### **Código Python**
```python
# Consultar a tabela registrada no catálogo
result_df = spark.sql("SELECT * FROM bitcoin_price")

# Mostrar os resultados
result_df.show()
```

---

### **2. Consulta Usando a API de DataFrame**
Se a tabela **não foi registrada** no catálogo e está apenas no **Delta Lake** no caminho `/mnt/delta/bitcoin_price`, você pode acessar diretamente usando o método `spark.read`.

#### **Código Python**
```python
# Carregar os dados diretamente do Delta Lake
result_df = spark.read.format("delta").load("/mnt/delta/bitcoin_price")

# Mostrar os resultados
result_df.show()
```

---

### **3. Diferença Entre os Métodos**
- **Usando SQL (`spark.sql`)**:
  - É possível consultar tabelas que foram registradas no catálogo.
  - Simples e adequado para quem prefere trabalhar com SQL.

- **Usando a API DataFrame (`spark.read.format`)**:
  - Útil para acessar diretamente os dados em caminhos físicos (como `/mnt/delta/...`).
  - Flexível para pipelines que não requerem registro de tabela.

---

### **Resumo**
1. Use **`spark.sql`** para consultar tabelas registradas no catálogo:
   ```python
   spark.sql("SELECT * FROM bitcoin_price").show()
   ```

2. Use **`spark.read.format("delta")`** para acessar diretamente os dados:
   ```python
   spark.read.format("delta").load("/mnt/delta/bitcoin_price").show()
   ```

Escolha a abordagem que melhor se adapta ao seu caso! Se precisar de ajuda com mais consultas ou análises, é só avisar. 😊

In [0]:
df = spark.read.format("delta").load("/mnt/delta/bitcoin_price")
df.show()

In [0]:
df = spark.table("default.bitcoin_price")
df.show()


In [0]:
%sql
SELECT * FROM default.bitcoin_price

In [0]:
%sql
SELECT count(*) FROM raw_transactions

In [0]:
%sql
SELECT 
    DATE(transaction_date) AS transaction_day,
    SUM(usd_value) AS total_transaction_volume_usd,
    SUM(btc_amount) AS total_transaction_volume_btc,
    COUNT(*) AS transaction_count
FROM transactions
GROUP BY DATE(transaction_date)
ORDER BY transaction_day;


In [0]:
%sql
SELECT 
    t.transaction_id,
    t.transaction_date,
    t.customer_id,
    t.btc_amount,
    t.usd_value,
    b.amount AS bitcoin_price_at_transaction,
    (t.usd_value / t.btc_amount) AS actual_price_per_btc
FROM transactions t
JOIN bitcoin_price b
ON DATE(t.transaction_date) = DATE(b.datetime)
   AND HOUR(t.transaction_date) = HOUR(b.datetime)
ORDER BY t.transaction_date;
